# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [19]:
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

import os

# Rutas de datasets y resultados
RUTA_DATASETS_BASE = "../datasets/datasets_aumentados/base/"
RUTA_DATASETS_AUMENTADOS = "../datasets/datasets_aumentados/"
RUTA_DATASETS_CLASICOS = "../datasets/datasets_aumentados/resampler_clasicos/"
DIRECTORIO_SALIDA = "../resultados"

os.makedirs(DIRECTORIO_SALIDA, exist_ok=True)
os.makedirs(RUTA_DATASETS_CLASICOS, exist_ok=True)


In [20]:
import gc, time  # gc: liberación explícita de memoria entre ejecuciones; time: medición de duración de búsquedas
from dataclasses import dataclass, asdict  # dataclass: estructura limpia para registrar resultados y metadatos de cada combinación
import json  # guardar resultados intermedios en formato JSON

import numpy as np  # operaciones numéricas y manipulación de vectores/matrices
import pandas as pd  # manejo de estructuras tabulares (dataframes) para consolidar resultados


# Utilizamos validación estratificada + búsqueda aleatoria de hiperparámetros
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

# Métricas utilizadas en CV y test (todas macro para evitar sesgos por clase mayoritaria)
from sklearn.metrics import (
    f1_score,
    balanced_accuracy_score,
    recall_score,
    make_scorer
)

# Cada modelo se ejecuta dentro de un Pipeline para permitir transformaciones futuras
from sklearn.pipeline import Pipeline

# Modelo principal evaluado (Random Forest)
from sklearn.ensemble import RandomForestClassifier

# Suprimir warnings de convergencia innecesarios (SVM no se usa en esta fase)
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Controlar comportamiento en entornos con múltiples núcleos
# (evita paralelismo interno conflictivo con n_jobs de sklearn)
import os

# Estado aleatorio fijo para reproducibilidad entre ejecuciones
RANDOM_STATE = 42
       
# En Shuttle aumentado omitimos SVM por inestabilidad del ROC-AUC y límites computacionales
OMITIR_SVM_EN_SHUTTLE_AUMENTADO = True

# Archivo Excel consolidado con resultados CV y Test para todas las técnicas
NOMBRE_ARCHIVO_EXCEL = os.path.join(DIRECTORIO_SALIDA, "resultados_RS_cv_vs_test.xlsx")

In [21]:
from pathlib import Path
import re

# =========================
# Estructuras de datos
# =========================
@dataclass
class DatasetCombination:
    dataset_logico: str
    tipo_combination: str      # "base" | "clasico" | "pcsmote"
    ruta_train_csv: str
    ruta_test_csv: str

    tecnica_aumento: str = "base"

    # Parámetros genéricos (clásicos / base)
    valor_densidad: str | int | None = "--"
    valor_riesgo: str | int | None = "--"
    criterio_pureza: str | None = "--"

    # Parámetros PCSMOTE (si aplica)
    percentil_radio_distancia: str | int | None = "--"
    percentil_riesgo: str | int | None = "--"
    umbral_densidad: str | None = "--"
    umbral_riesgo: str | None = "--"

    tipo_pureza: str = "--"           # PE.. o Upp.. del nombre de archivo
    nombre_configuracion: str = ""    # PRD.._PR.._CP.._UD.._UR.._.._I.._SV.._SG..

    grado_limpieza: str | int = "--"  # I0, I1, I3, etc.
    total_muestras_train: int | None = None
    tamanio_dataset: int | None = None  # tamaño total train+test (si lo tenés)
    sinteticos_generados: int = 0
    semillas_validas: int = 0


@dataclass
class RegistroRendimiento:
    dataset_logico: str
    tipo_combination: str
    nombre_modelo_aprendizaje: str
    nombre_configuracion: str
    tecnica_aumento: str
    valor_densidad: str
    valor_riesgo: str
    criterio_pureza: str
    grado_limpieza: str

    cantidad_train: int
    cantidad_test: int
    cantidad_caracteristicas: int

    # Métricas CV
    cv_f1_macro: float
    cv_balanced_accuracy: float
    cv_recall_macro: float

    # Métricas Test
    test_f1_macro: float
    test_balanced_accuracy: float
    test_recall_macro: float

    mejores_hiperparametros: str
    tiempo_busqueda_seg: float




def enumerar_combinaciones_base_y_aumentadas(
    ruta_base,
    ruta_clasicos,
    ruta_aumentados,
    verbose=True
):
    combinaciones = []
    cont_combinaciones = 0

    # Mapear (dataset_logico, grado_limpieza) → tamaño_train_base
    tamanio_train_base_por_dataset_y_I = {}

    # ==========================================================
    # 1) BASE
    #    train: {dataset}_I{I}_tm{n}_train.csv
    #    test : {dataset}_tm{n}_test.csv
    # ==========================================================
    if verbose:
        print(f"📂 Explorando carpeta base: {ruta_base}")

    archivos_base = os.listdir(ruta_base)

    for nombre in archivos_base:
        if not nombre.endswith("_train.csv"):
            if verbose:
                print(f"  ⚪ Omitido (no es *_train.csv): {nombre}")
            continue

        m = re.match(r"(.+?)_I(\d+)_tm(\d+)_train\.csv$", nombre)
        if not m:
            if verbose:
                print(f"  ⚪ No coincide patrón base con I*_tm*_train: {nombre}")
            continue

        dataset_logico = m.group(1)
        grado_limpieza = int(m.group(2))
        total_muestras_train = int(m.group(3))

        # Registrar tamaño de train base para este (dataset, I)
        clave_base = (dataset_logico, grado_limpieza)
        tamanio_train_base_por_dataset_y_I[clave_base] = total_muestras_train

        ruta_train_csv = os.path.join(ruta_base, nombre)

        # Buscar test correspondiente: {dataset}_tm{n}_test.csv
        patron_test = re.compile(rf"^{re.escape(dataset_logico)}_tdataset(\d+)_tm(\d+)_test\.csv$")
        nombre_test = None
        n_test_detectado = None

        for nombre_candidato in archivos_base:
            m_test = patron_test.match(nombre_candidato)
            if m_test:
                # Si hubiera más de uno, nos quedamos con el de mayor tm
                n_tm = int(m_test.group(2))
                tamanio_dataset=int(m_test.group(1))

                if n_test_detectado is None or n_tm > n_test_detectado:
                    n_test_detectado = n_tm
                    nombre_test = nombre_candidato

        if nombre_test is None:
            if verbose:
                print(f"  ⚠️  Falta test para dataset base '{dataset_logico}', se omite {nombre}")
            continue

        ruta_test_csv = os.path.join(ruta_base, nombre_test)

        cont_combinaciones += 1
        print(f"#{cont_combinaciones}  ✅ Agregado base: {nombre} combinado con {nombre_test}")

        combinaciones.append(DatasetCombination(
            dataset_logico=dataset_logico,
            tipo_combination="base",
            ruta_train_csv=ruta_train_csv,
            ruta_test_csv=ruta_test_csv,
            tecnica_aumento="base",
            valor_densidad=None,
            valor_riesgo=None,
            criterio_pureza=None,
            grado_limpieza=grado_limpieza,
            total_muestras_train=total_muestras_train,
            tamanio_dataset=tamanio_dataset
        ))

    # ==========================================================
    # 2) CLÁSICOS
    #    {tecnica}_{dataset}_I{I}_sg{sg}_train.csv
    #    test base: {dataset}_tm{n}_test.csv (mismo criterio que base)
    # ==========================================================
    if verbose:
        print(f"📂 Explorando carpeta clásicos: {ruta_clasicos}")

    archivos_clasicos = os.listdir(ruta_clasicos)

    for nombre in archivos_clasicos:
        if not nombre.endswith("_train.csv"):
            continue

        # ejemplo: adasyn_us_crime_I1_sg120_train.csv
        m = re.match(r"(.+?)_(.+?)_I(\d+)_sg(\d+)_train\.csv$", nombre)
        if not m:
            if verbose:
                print(f"  ⚠️  No cumple patrón clásicos: {nombre}")
            continue

        tecnica = m.group(1)
        dataset_logico = m.group(2)
        grado_limpieza = int(m.group(3))
        sinteticos_generados = int(m.group(4))

        # Recuperar tamaño de train base para este dataset y este I
        clave_base = (dataset_logico, grado_limpieza)
        total_muestras_train = tamanio_train_base_por_dataset_y_I.get(clave_base)

        if total_muestras_train is None:
            if verbose:
                print(
                    f"  ⚠️  No se encontró tamaño de train base para "
                    f"(dataset='{dataset_logico}', I={grado_limpieza}). Se omite {nombre}"
                )
            continue
        
        ruta_train_csv = os.path.join(ruta_clasicos, nombre)

        # Buscar test correspondiente en carpeta base
        patron_test = re.compile(rf"^{re.escape(dataset_logico)}_tdataset(\d+)_tm(\d+)_test\.csv$")
        nombre_test = None
        n_test_detectado = None

        for nombre_candidato in archivos_base:
            m_test = patron_test.match(nombre_candidato)
            if m_test:
                n_tm = int(m_test.group(2))
                tamanio_dataset = int(m_test.group(1))
                if n_test_detectado is None or n_tm > n_test_detectado:
                    n_test_detectado = n_tm
                    nombre_test = nombre_candidato

        if nombre_test is None:
            if verbose:
                print(f"  ⚠️  No hay test base para dataset '{dataset_logico}', se omite {nombre}")
            continue

        ruta_test_csv = os.path.join(ruta_base, nombre_test)

        cont_combinaciones += 1
        print(f"#{cont_combinaciones}  ✅ Agregado clásico: {nombre} combinado con {nombre_test}")

        combinaciones.append(DatasetCombination(
            dataset_logico=dataset_logico,
            tipo_combination="clasico",
            ruta_train_csv=ruta_train_csv,
            ruta_test_csv=ruta_test_csv,
            tecnica_aumento=tecnica.lower(),
            valor_densidad=None,
            valor_riesgo=None,
            criterio_pureza=None,
            grado_limpieza=grado_limpieza,
            total_muestras_train=total_muestras_train,
            sinteticos_generados=sinteticos_generados,
            tamanio_dataset=tamanio_dataset
        ))

    # ==========================================================
    # 3) PC-SMOTE (nuevo patrón)
    #
    # pcs_{dataset}_PRD{prd}_PR{pr}_CP{ent|prop}_UD{ud3}_{PE..|Ppp..}_I{iso}_SG{sg}_train.csv
    #
    # Ej:
    #   pcs_ecoli_PRD35_PR35_CPent_UD080_PE45_I0_SG120_train.csv
    #   pcs_ecoli_PRD35_PR35_CPprop_UD080_Ppp041_I0_SG007_train.csv
    #
    # valor_densidad  → percentil radio distancia (PRD)
    # valor_riesgo    → percentil riesgo (PR)
    # criterio_pureza → "entropia" / "proporcion"
    # grado_limpieza  → iso (I*)
    # sinteticos_generados → SG
    # ==========================================================
    if verbose:
        print(f"📂 Explorando carpeta aumentados: {ruta_aumentados}")

    archivos_aumentados = os.listdir(ruta_aumentados)

    patron_pcsmote = re.compile(
        r"^pcs_(?P<dataset>.+?)_"
        r"PRD(?P<prd>\d+)_"
        r"PR(?P<pr>\d+)_"
        r"CP(?P<cp>(?:ent|prop))_"
        r"UD(?P<ud>\d{3})_"
        r"(?P<tipo_pureza>(?:PE\d+|Upp\d{3}))_"
        r"UR(?P<ur>\d{3})_"
        r"I(?P<iso>\d+)_"
        r"SV(?P<sv>\d+)_"
        r"SG(?P<sg>\d+)_train\.csv$"
    )

    for nombre in archivos_aumentados:
        if not nombre.endswith("_train.csv"):
            continue

        m = patron_pcsmote.match(nombre)
        if not m:
            if verbose:
                print(f"  ⚪ Omitido (no es pcs válido): {nombre}")
            continue

        dataset_logico = m.group("dataset")
        valor_densidad = int(m.group("prd"))   # percentil radio distancia
        valor_riesgo   = int(m.group("pr"))    # percentil riesgo
        cp_code        = m.group("cp")         # "ent" | "prop"
        ud_str       = m.group("ud")        # umbral densidad en %, si después lo querés usar
        ur_str       = m.group("ur")        # umbral densidad en %, si después lo querés usar
        tipo_pureza = m.group("tipo_pureza")  # PE.. / Ppp.., si lo necesitás luego
        grado_limpieza = int(m.group("iso"))   # I*
        semillas_validas = int(m.group("sv"))
        sinteticos_generados = int(m.group("sg"))

        print(f"  ➡️  Descifrado pcsmote: dataset={dataset_logico}, prd={valor_densidad}, pr={valor_riesgo}, cp={cp_code}, ud={ud_str}, ur={ur_str}, tipo_pureza={tipo_pureza}, I={grado_limpieza}, sv={semillas_validas}, sg={sinteticos_generados}")

        if cp_code == "ent":
            criterio_pureza = "entropia"
        else:
            criterio_pureza = "proporcion"

        # nombre_configuracion EXACTO según el patrón
        nombre_configuracion = (
            f"PRD{valor_densidad}_"
            f"PR{valor_riesgo}_"
            f"CP{cp_code}_"
            f"UD{ud_str}_"
            f"UR{ur_str}_"
            f"{tipo_pureza}_"
            f"I{grado_limpieza}_"
            f"SV{semillas_validas}_"
            f"SG{sinteticos_generados}"
        )            

        ruta_train_csv = os.path.join(ruta_aumentados, nombre)

        # Buscar test correspondiente en carpeta base
        patron_test = re.compile(rf"^{re.escape(dataset_logico)}_tdataset(\d+)_tm(\d+)_test\.csv$")
        nombre_test = None
        n_test_detectado = None

        for nombre_candidato in archivos_base:
            m_test = patron_test.match(nombre_candidato)
            if m_test:
                n_tm = int(m_test.group(2))
                tamanio_dataset = int(m_test.group(1))
                if n_test_detectado is None or n_tm > n_test_detectado:
                    n_test_detectado = n_tm
                    nombre_test = nombre_candidato

        if nombre_test is None:
            if verbose:
                print(f"  ⚠️  No hay test base para dataset '{dataset_logico}', se omite {nombre}")
            continue

        ruta_test_csv = os.path.join(ruta_base, nombre_test)

        cont_combinaciones += 1
        print(f"#{cont_combinaciones}  ✅ Agregado pcsmote: {nombre} combinado con {nombre_test}")

        combinaciones.append(DatasetCombination(
            dataset_logico=dataset_logico,
            tipo_combination="pcsmote",
            ruta_train_csv=ruta_train_csv,
            ruta_test_csv=ruta_test_csv,
            tecnica_aumento="pcsmote",
            valor_densidad=valor_densidad,
            valor_riesgo=valor_riesgo,

            criterio_pureza=criterio_pureza,
            percentil_radio_distancia=valor_densidad,
            percentil_riesgo=valor_riesgo,  
            umbral_densidad=ud_str,
            umbral_riesgo=ur_str,

            grado_limpieza=grado_limpieza,
            sinteticos_generados=sinteticos_generados,
            semillas_validas=semillas_validas,
            tipo_pureza=tipo_pureza,                 
            nombre_configuracion=nombre_configuracion,    
            tamanio_dataset=tamanio_dataset
    
        ))

    if verbose:
        print(f"📊 Total combinaciones descubiertas: {len(combinaciones)}")

    return combinaciones


print("🔎 Enumerando combinaciones base y aumentadas...")

combinaciones = enumerar_combinaciones_base_y_aumentadas(
    ruta_base=RUTA_DATASETS_BASE,
    ruta_clasicos=RUTA_DATASETS_CLASICOS,
    ruta_aumentados=RUTA_DATASETS_AUMENTADOS,
    verbose=True
)

if not combinaciones:
    print("❌ No se encontraron combinaciones de datasets.")


datasets_con_base = {c.dataset_logico for c in combinaciones if c.tipo_combination == "base"}
if not datasets_con_base:
    print("❌ No hay datasets base para comparar.")



🔎 Enumerando combinaciones base y aumentadas...
📂 Explorando carpeta base: ../datasets/datasets_aumentados/base/
#1  ✅ Agregado base: ecoli_I0_tm268_train.csv combinado con ecoli_tdataset336_tm68_test.csv
#2  ✅ Agregado base: ecoli_I1_tm262_train.csv combinado con ecoli_tdataset336_tm68_test.csv
#3  ✅ Agregado base: ecoli_I3_tm258_train.csv combinado con ecoli_tdataset336_tm68_test.csv
  ⚪ Omitido (no es *_train.csv): ecoli_tdataset336_tm68_test.csv
#4  ✅ Agregado base: gear_vibration_I0_tm72_train.csv combinado con gear_vibration_tdataset90_tm18_test.csv
#5  ✅ Agregado base: gear_vibration_I1_tm68_train.csv combinado con gear_vibration_tdataset90_tm18_test.csv
#6  ✅ Agregado base: gear_vibration_I3_tm68_train.csv combinado con gear_vibration_tdataset90_tm18_test.csv
  ⚪ Omitido (no es *_train.csv): gear_vibration_tdataset90_tm18_test.csv
#7  ✅ Agregado base: glass_I0_tm171_train.csv combinado con glass_tdataset214_tm43_test.csv
#8  ✅ Agregado base: glass_I1_tm166_train.csv combinado c

In [22]:
EXCLUIR_DATASETS = {
    "shuttle",
    "iris",
    # "glass",
    # "heart",
    # "wdbc",
    # "ecoli",
    # "us_crime",
    # "predict_faults",
    "gear_vibration",
    # "telco_churn",
}

def construir_lista_plana_de_tareas(dataset_combinations, orden_modelos,
                                    excluir_datasets=EXCLUIR_DATASETS, verbose=True):
    """
    Lista plana de tareas (modelo, combinación) aplicando exclusión por dataset.
    Nota: La política SVM/Shuttle no aplica acá porque esta notebook evalúa RandomForest.
    """
    tareas = []
    excluidos_por_dataset = 0

    for nombre_modelo in orden_modelos:
        for combo in dataset_combinations:
            ds = combo.dataset_logico.lower()

            if ds in (excluir_datasets or set()):
                excluidos_por_dataset += 1
                continue

            tareas.append((nombre_modelo, combo))

    if verbose:
        print(f"🧮 Tareas planificadas: {len(tareas)} (excluidos por dataset: {excluidos_por_dataset})")
    return tareas


def construir_estimador_y_espacio_random_forest():
    est = Pipeline([
        ('classifier', RandomForestClassifier(
            random_state=RANDOM_STATE,
            n_jobs=1,
            bootstrap=True,
            oob_score=False,
            n_estimators=150,
            max_depth=None,
            max_features='sqrt',
            min_samples_split=2,
            min_samples_leaf=1,
            class_weight=None,
            criterion='gini'
        ))
    ])

    # Espacio chico y controlado (4 combinaciones posibles)
    space = {
        "classifier__n_estimators": [150, 300],
        "classifier__max_features": ["sqrt", "log2"],
    }
    return est, space


REGISTRO_MODELOS = {
    "RandomForest": construir_estimador_y_espacio_random_forest,
}
ORDEN_MODELOS = ["RandomForest"]


tareas = construir_lista_plana_de_tareas(
    dataset_combinations=combinaciones,
    orden_modelos=ORDEN_MODELOS,
    excluir_datasets=EXCLUIR_DATASETS,
    verbose=True
)

total_tareas = len(tareas)
print(f"📦 Total de tareas planificadas: {total_tareas}")


🧮 Tareas planificadas: 1144 (excluidos por dataset: 133)
📦 Total de tareas planificadas: 1144


In [23]:
# Scoring para RandomizedSearchCV
SCORING_REFIT = "f1_macro"
SCORING_MULTIPLE = {
    "f1_macro": "f1_macro",
    "balanced_accuracy": "balanced_accuracy",
    "recall_macro": make_scorer(recall_score, average="macro"),
}


def _contar_combinaciones_posibles_en_space(space):
    """
    Cuenta combinaciones si space es un dict de listas discretas.
    Si encuentra algo que no es lista/tupla/np.ndarray, devuelve None.
    """
    total = 1
    for k, v in space.items():
        if isinstance(v, (list, tuple, np.ndarray)):
            total *= len(v)
        else:
            return None
    return total


def ejecutar_rs_y_comparar_cv_con_test(
    estimator,
    space,
    X_train,
    y_train,
    X_test,
    y_test,
    configuracion_busqueda,
    verbose=0,
):
    """
    Ejecuta RandomizedSearchCV y devuelve:
      - mejores params
      - tiempo
      - métricas CV del mejor candidato
      - métricas Test con best_estimator_
    """
    inicio = time.perf_counter()

    n_iter_solicitado = int(configuracion_busqueda["n_iter"])
    n_posibles = _contar_combinaciones_posibles_en_space(space)

    # Si el space es discreto (como tu RF), no tiene sentido pedir más iteraciones que combinaciones
    if n_posibles is not None:
        n_iter_efectivo = min(n_iter_solicitado, n_posibles)
    else:
        n_iter_efectivo = n_iter_solicitado

    search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=space,
        n_iter=n_iter_efectivo,
        scoring=SCORING_MULTIPLE,
        refit=SCORING_REFIT,
        cv=configuracion_busqueda["cv"],
        random_state=RANDOM_STATE,
        n_jobs=configuracion_busqueda["n_jobs"],
        verbose=verbose,
    )

    search.fit(X_train, y_train)
    elapsed = time.perf_counter() - inicio

    cv_results = search.cv_results_
    best_idx = search.best_index_

    cv_f1       = float(cv_results["mean_test_f1_macro"][best_idx])
    cv_bacc     = float(cv_results["mean_test_balanced_accuracy"][best_idx])
    cv_recall_m = float(cv_results["mean_test_recall_macro"][best_idx])

    best_est = search.best_estimator_
    y_pred = best_est.predict(X_test)

    test_f1       = float(f1_score(y_test, y_pred, average="macro"))
    test_bacc     = float(balanced_accuracy_score(y_test, y_pred))
    test_recall_m = float(recall_score(y_test, y_pred, average="macro"))

    return dict(
        mejores_params=search.best_params_,
        tiempo=float(elapsed),
        n_iter_efectivo=int(n_iter_efectivo),
        n_combinaciones_posibles=(int(n_posibles) if n_posibles is not None else None),
        cv=dict(
            f1=cv_f1,
            bacc=cv_bacc,
            recall_macro=cv_recall_m,
        ),
        test=dict(
            f1=test_f1,
            bacc=test_bacc,
            recall_macro=test_recall_m,
        ),
    )


In [24]:
N_ITER_BUSQUEDA_POR_DEFECTO = 4

# =========================
# Utilidades de datos
# =========================
def cargar_matriz_caracteristicas_y_etiquetas_desde_csv(ruta_csv):
    """Lee un CSV y devuelve (X, y). Usa 'target' si existe, si no la última columna como y."""
    df = pd.read_csv(ruta_csv)
    if "target" in df.columns:
        X = df.drop(columns=["target"]).to_numpy(dtype=np.float32, copy=False)
        y = df["target"].to_numpy()
    else:
        X = df.iloc[:, :-1].to_numpy(dtype=np.float32, copy=False)
        y = df.iloc[:, -1].to_numpy()
    return X, y


def definir_configuracion_busqueda_para_dataset(X_train, nombre_dataset_logico, tipo_combination):
    """
    Define configuración de búsqueda SOLO para el tuning de base.
    - Shuttle aumentado -> CV=2 (no aplica si solo tuneamos base)
    - Shuttle o n>=10000 -> CV=3
    - resto -> CV=5
    """
    n_muestras = X_train.shape[0]
    es_shuttle = nombre_dataset_logico.lower() == "shuttle"

    # Como tuneamos SOLO base, este branch normalmente no se usa.
    if es_shuttle and tipo_combination != "base":
        cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    elif es_shuttle or n_muestras >= 10000:
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    else:
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

    return {
        "cv": cv,
        "n_iter": N_ITER_BUSQUEDA_POR_DEFECTO,
        "n_jobs": 1,
    }


def entrenar_y_evaluar_con_params_fijos(
    estimator,
    mejores_params,
    X_train,
    y_train,
    X_test,
    y_test,
):
    """
    Entrena un estimador con hiperparámetros fijos y evalúa en Test.
    Retorna métricas (solo Test) y el tiempo de entrenamiento+predicción.
    """
    inicio = time.perf_counter()

    estimator.set_params(**mejores_params)
    estimator.fit(X_train, y_train)

    y_pred = estimator.predict(X_test)

    elapsed = time.perf_counter() - inicio

    test_f1 = float(f1_score(y_test, y_pred, average="macro"))
    test_bacc = float(balanced_accuracy_score(y_test, y_pred))
    test_recall_m = float(recall_score(y_test, y_pred, average="macro"))

    return dict(
        tiempo=float(elapsed),
        test=dict(
            f1=test_f1,
            bacc=test_bacc,
            recall_macro=test_recall_m,
        ),
    )


def _clave_contexto(combo, nombre_modelo):
    # clave por dataset + modelo + grado de limpieza (I)
    return (combo.dataset_logico.lower(), nombre_modelo, str(combo.grado_limpieza))


# =========================
# Paso 1: TUNING SOLO BASE
# =========================
best_params_por_contexto = {}
ruta_best_params = os.path.join(DIRECTORIO_SALIDA, "best_params_por_contexto.json")

print("\n" + "="*100)
print("🎯 ETAPA 1: TUNING SOLO EN 'BASE' (por dataset + modelo + I)")
print("="*100)

for idx, (nombre_modelo, combo) in enumerate(tareas, start=1):

    if combo.tipo_combination != "base":
        continue

    clave = _clave_contexto(combo, nombre_modelo)

    if clave in best_params_por_contexto:
        continue

    print(f"\n{'='*80}")
    print(f"🎯 TUNING BASE [{idx}/{total_tareas}] Dataset: {combo.dataset_logico} | "
          f"Modelo: {nombre_modelo} | I={combo.grado_limpieza}")
    print(f"📂 Train(base): {os.path.basename(combo.ruta_train_csv)}")
    print(f"📂 Test (base): {os.path.basename(combo.ruta_test_csv)}")

    # cargar base
    try:
        X_train, y_train = cargar_matriz_caracteristicas_y_etiquetas_desde_csv(combo.ruta_train_csv)
        X_test,  y_test  = cargar_matriz_caracteristicas_y_etiquetas_desde_csv(combo.ruta_test_csv)
    except Exception as e:
        print(f"❌ Error leyendo CSV base: {e}")
        continue

    configuracion_busqueda = definir_configuracion_busqueda_para_dataset(
        X_train, combo.dataset_logico, combo.tipo_combination
    )

    estimator, space = REGISTRO_MODELOS[nombre_modelo]()

    print(f"⚙️  RS(base): n_iter={configuracion_busqueda['n_iter']} (auto-ajustado dentro de ejecutar_rs), "
          f"folds={configuracion_busqueda['cv'].n_splits}, n_jobs={configuracion_busqueda['n_jobs']}")
    print("🚀 Iniciando RandomizedSearchCV en BASE...")

    try:
        resultados = ejecutar_rs_y_comparar_cv_con_test(
            estimator, space, X_train, y_train, X_test, y_test,
            configuracion_busqueda=configuracion_busqueda,
            verbose=1
        )
    except Exception as e:
        print(f"❌ Error durante RS(base): {e}")
        continue

    best_params_por_contexto[clave] = resultados["mejores_params"]

    # guardado incremental por si se corta
    try:
        serializable = {str(k): v for k, v in best_params_por_contexto.items()}
        with open(ruta_best_params, "w", encoding="utf-8") as f:
            json.dump(serializable, f, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"⚠️ No se pudo guardar best_params JSON: {e}")

    print(f"✅ RS(base) completado en {resultados['tiempo']:.2f} s | n_iter efectivo={resultados.get('n_iter_efectivo', '--')}")
    print(f"📊 BASE F1(CV): {resultados['cv']['f1']:.4f} | F1(Test): {resultados['test']['f1']:.4f}")
    print(f"🧠 Best params: {resultados['mejores_params']}")


if not best_params_por_contexto:
    raise RuntimeError("❌ No se obtuvo ningún best_params (no se encontró base o falló el tuning).")


# =========================
# Paso 2: EVALUACIÓN MASIVA con params fijos
# =========================
registros = []
inicio_total = time.perf_counter()

print("\n" + "="*100)
print("🏁 ETAPA 2: EVALUACIÓN MASIVA (params fijos) sobre TODOS los train CSV generados")
print("="*100)

for idx, (nombre_modelo, combo) in enumerate(tareas, start=1):

    print(f"\n{'='*80}")
    print(f"🏁 [{idx}/{total_tareas}] Dataset: {combo.dataset_logico} | "
          f"Tipo: {combo.tipo_combination} | Modelo: {nombre_modelo}")
    print(f"📂 Train: {os.path.basename(combo.ruta_train_csv)}")

    clave = _clave_contexto(combo, nombre_modelo)
    if clave not in best_params_por_contexto:
        print(f"⚠️ Sin best_params para {clave}. Se omite.")
        continue

    mejores_params = best_params_por_contexto[clave]

    # cargar train/test (de cada combinación)
    try:
        X_train, y_train = cargar_matriz_caracteristicas_y_etiquetas_desde_csv(combo.ruta_train_csv)
        X_test,  y_test  = cargar_matriz_caracteristicas_y_etiquetas_desde_csv(combo.ruta_test_csv)
    except Exception as e:
        print(f"❌ Error leyendo CSV: {e}")
        continue

    estimator, _space = REGISTRO_MODELOS[nombre_modelo]()

    # entrenar + evaluar (sin RS)
    try:
        resultados = entrenar_y_evaluar_con_params_fijos(
            estimator=estimator,
            mejores_params=mejores_params,
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
        )
    except Exception as e:
        print(f"❌ Error durante fit/eval params fijos: {e}")
        continue

    print(f"✅ Eval completada en {resultados['tiempo']:.2f} s")
    print(f"📊 F1(Test): {resultados['test']['f1']:.4f}")

    # registrar resultados (CV ya no aplica por combinación: NaN)
    registros.append(asdict(RegistroRendimiento(
        dataset_logico=combo.dataset_logico,
        tipo_combination=combo.tipo_combination,
        nombre_modelo_aprendizaje=nombre_modelo,
        nombre_configuracion=combo.nombre_configuracion,
        tecnica_aumento=combo.tecnica_aumento,
        valor_densidad=str(combo.valor_densidad),
        valor_riesgo=str(combo.valor_riesgo),
        criterio_pureza=str(combo.criterio_pureza),
        grado_limpieza=str(combo.grado_limpieza),

        cantidad_train=int(X_train.shape[0]),
        cantidad_test=int(X_test.shape[0]),
        cantidad_caracteristicas=int(X_train.shape[1]),

        cv_f1_macro=float("nan"),
        cv_balanced_accuracy=float("nan"),
        cv_recall_macro=float("nan"),

        test_f1_macro=round(resultados["test"]["f1"], 3),
        test_balanced_accuracy=round(resultados["test"]["bacc"], 3),
        test_recall_macro=round(resultados["test"]["recall_macro"], 3),

        mejores_hiperparametros=str(mejores_params),
        tiempo_busqueda_seg=float(resultados["tiempo"]),  # ahora es tiempo fit+pred
    )))

    gc.collect()

fin_total = time.perf_counter() - inicio_total
print(f"\n⏱️ Tiempo total etapa train/test (params fijos): {fin_total/60:.2f} min")



🎯 ETAPA 1: TUNING SOLO EN 'BASE' (por dataset + modelo + I)

🎯 TUNING BASE [1/1144] Dataset: ecoli | Modelo: RandomForest | I=0
📂 Train(base): ecoli_I0_tm268_train.csv
📂 Test (base): ecoli_tdataset336_tm68_test.csv
⚙️  RS(base): n_iter=4 (auto-ajustado dentro de ejecutar_rs), folds=5, n_jobs=1
🚀 Iniciando RandomizedSearchCV en BASE...
Fitting 5 folds for each of 4 candidates, totalling 20 fits


c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes n

✅ RS(base) completado en 14.02 s | n_iter efectivo=4
📊 BASE F1(CV): 0.7013 | F1(Test): 0.8846
🧠 Best params: {'classifier__n_estimators': 150, 'classifier__max_features': 'sqrt'}

🎯 TUNING BASE [2/1144] Dataset: ecoli | Modelo: RandomForest | I=1
📂 Train(base): ecoli_I1_tm262_train.csv
📂 Test (base): ecoli_tdataset336_tm68_test.csv
⚙️  RS(base): n_iter=4 (auto-ajustado dentro de ejecutar_rs), folds=5, n_jobs=1
🚀 Iniciando RandomizedSearchCV en BASE...
Fitting 5 folds for each of 4 candidates, totalling 20 fits


c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes n

✅ RS(base) completado en 15.42 s | n_iter efectivo=4
📊 BASE F1(CV): 0.7104 | F1(Test): 0.8846
🧠 Best params: {'classifier__n_estimators': 150, 'classifier__max_features': 'sqrt'}

🎯 TUNING BASE [3/1144] Dataset: ecoli | Modelo: RandomForest | I=3
📂 Train(base): ecoli_I3_tm258_train.csv
📂 Test (base): ecoli_tdataset336_tm68_test.csv
⚙️  RS(base): n_iter=4 (auto-ajustado dentro de ejecutar_rs), folds=5, n_jobs=1
🚀 Iniciando RandomizedSearchCV en BASE...
Fitting 5 folds for each of 4 candidates, totalling 20 fits


c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes n

✅ RS(base) completado en 13.68 s | n_iter efectivo=4
📊 BASE F1(CV): 0.7048 | F1(Test): 0.8846
🧠 Best params: {'classifier__n_estimators': 150, 'classifier__max_features': 'sqrt'}

🎯 TUNING BASE [4/1144] Dataset: glass | Modelo: RandomForest | I=0
📂 Train(base): glass_I0_tm171_train.csv
📂 Test (base): glass_tdataset214_tm43_test.csv
⚙️  RS(base): n_iter=4 (auto-ajustado dentro de ejecutar_rs), folds=5, n_jobs=1
🚀 Iniciando RandomizedSearchCV en BASE...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ RS(base) completado en 12.88 s | n_iter efectivo=4
📊 BASE F1(CV): 0.6739 | F1(Test): 0.8374
🧠 Best params: {'classifier__n_estimators': 150, 'classifier__max_features': 'sqrt'}

🎯 TUNING BASE [5/1144] Dataset: glass | Modelo: RandomForest | I=1
📂 Train(base): glass_I1_tm166_train.csv
📂 Test (base): glass_tdataset214_tm43_test.csv
⚙️  RS(base): n_iter=4 (auto-ajustado dentro de ejecutar_rs), folds=5, n_jobs=1
🚀 Iniciando RandomizedSearchCV en BASE...
Fitting 5 folds for each of 

In [ ]:
# =========================
# EXPORT FINAL A EXCEL (LIMPIO + snake_case) + DEBUG
# =========================
DEBUG_EXPORT = True
DEBUG_MAX_PRINT = 50   # para no spamear consola
debug_prints = 0
debug_stats = {
    "pcsmote": {"ok": 0, "miss": 0},
    "clasico": {"ok": 0, "miss": 0, "filtro_subtecnica_aplicado": 0},
    "base": {"ok": 0, "miss": 0},
}

inicio_total = time.perf_counter()

# 1) Consolidar registros a DataFrame
df_resultados = pd.DataFrame(registros)
df_resultados = df_resultados.copy()
df_resultados.reset_index(drop=True, inplace=True)

if DEBUG_EXPORT:
    print("=== DEBUG EXPORT ===")
    print(f"registros: {len(registros)} | combinaciones: {len(combinaciones)}")
    print(f"df_resultados.shape: {df_resultados.shape}")
    print(f"columnas df_resultados: {list(df_resultados.columns)}")
    print("====================")

# 2) Normalizar tipos (solo lo que se exporta)
for col in ["test_f1_macro", "test_balanced_accuracy", "test_recall_macro"]:
    if col in df_resultados.columns:
        df_resultados[col] = pd.to_numeric(df_resultados[col], errors="coerce")

# 4) Resumen para export (snake_case)
resultados_formateados = []

for idx, comb in enumerate(combinaciones, start=1):

    tecnica_comb = comb.tecnica_aumento
    dataset_comb = comb.dataset_logico
    grado_comb = comb.grado_limpieza

    print(f"combinacion meta:{comb.tecnica_aumento}")

    # 1) PC-SMOTE: match exacto por nombre_configuracion
    if tecnica_comb == "pcsmote":
        clave = comb.nombre_configuracion
        df_fila = df_resultados[
            (df_resultados["tecnica_aumento"] == "pcsmote") &
            (df_resultados["nombre_configuracion"] == clave)
        ]
        rama = "pcsmote"

    # 2) CLÁSICOS
    elif tecnica_comb == "clasico":
        df_fila = df_resultados[
            (df_resultados["tecnica_aumento"] == "clasico") &
            (df_resultados["dataset_logico"] == dataset_comb) &
            (df_resultados["grado_limpieza"] == grado_comb)
        ]
        rama = "clasico"

    # 3) BASE
    else:
        
        df_fila = df_resultados[
            (df_resultados["tecnica_aumento"] == "base") &
            (df_resultados["dataset_logico"] == dataset_comb) &
            (df_resultados["grado_limpieza"] == grado_comb)
        ]
        rama = "base"

    debug_stats[rama]["ok"] += 1

    # si hay más de una (posibles re-ejecuciones), tomamos la última registrada
    fila = df_fila.iloc[-1]

    if DEBUG_EXPORT and debug_prints < DEBUG_MAX_PRINT:
        subtecnica_fila = fila.get("subtecnica", "") if "subtecnica" in df_resultados.columns else ""
        print(
            f"[{idx}/{len(combinaciones)}] OK {rama.upper()} | ds={dataset_comb} I={grado_comb} "
            f"| matches={len(df_fila)} | f1={fila.get('test_f1_macro', None)} | sub={subtecnica_fila}"
        )
        debug_prints += 1

    tecnica = str(fila.get("tecnica_aumento", "base"))

    # Valores numéricos administrativos (base -> NaN)
    if tecnica in ("pcsmote", "clasico"):
        semillas_candidatas = getattr(comb, "semillas_validas", np.nan)
        sinteticos_generados = getattr(comb, "sinteticos_generados", np.nan)
    else:
        semillas_candidatas = np.nan
        sinteticos_generados = np.nan

    resultados_formateados.append({
        "tecnica": tecnica,
        "dataset": comb.dataset_logico,

        "percentil_radio_distancia": getattr(comb, "percentil_radio_distancia", "--"),
        "percentil_riesgo": getattr(comb, "percentil_riesgo", "--"),
        "criterio_pureza": getattr(comb, "criterio_pureza", "--"),
        "umbral_densidad": getattr(comb, "umbral_densidad", "--"),
        "umbral_riesgo": getattr(comb, "umbral_riesgo", "--"),
        "tipo_pureza": getattr(comb, "tipo_pureza", "--"),

        "total_n": getattr(comb, "tamanio_dataset", np.nan),
        "grado_isolation_forest": str(comb.grado_limpieza),
        "total_isolation": getattr(comb, "total_muestras_train", np.nan),

        "test_n": int(fila.get("cantidad_test", 0)),
        "train_n": int(fila.get("cantidad_train", 0)),

        "semillas_candidatas_train": float(semillas_candidatas) if pd.notna(semillas_candidatas) else np.nan,
        "sinteticas_generadas": float(sinteticos_generados) if pd.notna(sinteticos_generados) else np.nan,

        "f1_macro_test": float(fila.get("test_f1_macro", np.nan)) if pd.notna(fila.get("test_f1_macro", np.nan)) else np.nan,
        "balanced_accuracy_test": float(fila.get("test_balanced_accuracy", np.nan)) if pd.notna(fila.get("test_balanced_accuracy", np.nan)) else np.nan,
        "recall_macro_test": float(fila.get("test_recall_macro", np.nan)) if pd.notna(fila.get("test_recall_macro", np.nan)) else np.nan,

        "mejores_hiperparametros": fila.get("mejores_hiperparametros", ""),
    })

if DEBUG_EXPORT:
    print("=== DEBUG RESUMEN ===")
    print(debug_stats)
    print(f"filas exportadas (resultados_formateados): {len(resultados_formateados)}")
    print("=====================")

# 5) Export a Excel
df_export = pd.DataFrame(resultados_formateados)

ruta_excel = NOMBRE_ARCHIVO_EXCEL
df_export.to_excel(ruta_excel, index=False)

fin_total = time.perf_counter()
duracion = round(fin_total - inicio_total, 2)

print(f"✅ Export Excel final OK: {ruta_excel}")
print(f"📊 Filas exportadas: {len(df_export)}")
print(f"⏱️ Duración export: {duracion} seg")


=== DEBUG EXPORT ===
registros: 1144 | combinaciones: 1277
df_resultados.shape: (1144, 20)
columnas df_resultados: ['dataset_logico', 'tipo_combination', 'nombre_modelo_aprendizaje', 'nombre_configuracion', 'tecnica_aumento', 'valor_densidad', 'valor_riesgo', 'criterio_pureza', 'grado_limpieza', 'cantidad_train', 'cantidad_test', 'cantidad_caracteristicas', 'cv_f1_macro', 'cv_balanced_accuracy', 'cv_recall_macro', 'test_f1_macro', 'test_balanced_accuracy', 'test_recall_macro', 'mejores_hiperparametros', 'tiempo_busqueda_seg']
combinacion meta:base
[1/1277] MISS BASE | ds=ecoli I=0
combinacion meta:base
[2/1277] MISS BASE | ds=ecoli I=1
combinacion meta:base
[3/1277] MISS BASE | ds=ecoli I=3
combinacion meta:base
[4/1277] MISS BASE | ds=gear_vibration I=0
combinacion meta:base
[5/1277] MISS BASE | ds=gear_vibration I=1
combinacion meta:base
[6/1277] MISS BASE | ds=gear_vibration I=3
combinacion meta:base
[7/1277] MISS BASE | ds=glass I=0
combinacion meta:base
[8/1277] MISS BASE | ds=gla